In [22]:
!pip install yfinance
# !pip install pygraphviz


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
#openAI Key

In [3]:
from langchain_core.tools import tool, StructuredTool
import yfinance as yf

@tool
def company_address(ticker: str) -> str:
    """
    Returns company address for input ticker.
    e.g. company_address: AAPL
    Returns company address for ticker AAPL which is stock ticker for Apple Inc.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return " ".join([info[key] for key in ['address1','city','state','zip','country']])

@tool
def fulltime_employees(ticker: str) -> int:
    """
    Returns fulltime employees count for input ticker.
    e.g. company_address: MSFT
    Returns fulltime employees count for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return info['fullTimeEmployees']

@tool
def last_close_price(ticker: str) -> float:
    """
    Returns last close price for input ticker.
    e.g. company_address: MSFT
    Returns last close price for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return info['previousClose']

@tool
def EBITDA(ticker: str) -> float:
    """
    Returns EBITDA for input ticker.
    e.g. company_address: AAPL
    Returns EBITDA for ticker AAPL which is stock ticker for Apple Inc.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return info['ebitda']

@tool
def total_debt(ticker: str) -> float:
    """
    Returns total debt for input ticker.
    e.g. company_address: AAPL
    Returns total debt for ticker AAPL which is stock ticker for Apple Inc.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return info['totalDebt']

@tool
def total_revenue(ticker: str) -> float:
    """
    Returns total revenue for input ticker.
    e.g. company_address: MSFT
    Returns total revenue for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return info['totalRevenue']

@tool
def debt_to_equity_ratio(ticker: str) -> float:
    """
    Returns debt to equity ratio for input ticker.
    e.g. company_address: AAPL
    Returns debt to equity ratio for ticker AAPL which is stock ticker for Apple Inc.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return info['debtToEquity']

tools = [
    company_address,
    fulltime_employees,
    last_close_price,
    EBITDA,
    total_debt,
    total_revenue,
    debt_to_equity_ratio
]

In [4]:
llm_with_tools = llm.bind_tools(tools, tool_choice='auto')

In [5]:
ai_msg = llm_with_tools.invoke('Whatzzz up??')

print(ai_msg.content)
print(ai_msg.tool_calls)

Hello! I'm here to help you with any questions or information you need. How can I assist you today?
[]


In [6]:
ai_msg = llm_with_tools.invoke("What's the EBITDA of Microsoft?")

print(ai_msg.content)
print(ai_msg.tool_calls)



[{'name': 'EBITDA', 'args': {'ticker': 'MSFT'}, 'id': 'call_zGKBFhTUBKnKhmrfE5r3rX97', 'type': 'tool_call'}]


In [7]:
EBITDA.invoke(ai_msg.tool_calls[0]['args'])

142041006080

In [8]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import HumanMessage, AnyMessage, SystemMessage, ToolMessage

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

class ReActAgent:
    def __init__(self, model, tools, system=""):
        self.system = system
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools, tool_choice="auto")

        graph = StateGraph(AgentState)
        graph.add_node("llama3", self.call_llm)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llama3",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llama3")
        graph.set_entry_point("llama3")
        self.graph = graph.compile()

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_llm(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling Tool : {t}")
            if not t['name'] in self.tools:
                print(f"\n Tool : {t} does not exist.")
                result = "Incorrect Tool Name, Please retry and select tool from available tools."
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id = t['id'], name = t['name'], content = str(result)))
        print("Tool execution is completed. Back to the model!")
        return {'messages' : results}

In [9]:
prompt = """
You are a smart AI finance assistant. Use the list of available tools to answer questions if needed.
You are allowed to make multiple calls (either together or in sequence).
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

agent = ReActAgent(llm, tools, system=prompt)

In [11]:
messages = [HumanMessage(content="What's up Llama 3.1?")]

result = agent.graph.invoke({'messages': messages})

result['messages'][-1].content

"Hello! I'm here to assist you with any financial inquiries or tasks you have. How can I help you today?"

In [17]:
messages = [HumanMessage(content="What is the EBITDA of Target and its last close price?")]

result = agent.graph.invoke({'messages': messages})

Calling Tool : {'name': 'EBITDA', 'args': {'ticker': 'TGT'}, 'id': 'call_eKBYzK5uincTMXvCkv6ByZy5', 'type': 'tool_call'}
Calling Tool : {'name': 'last_close_price', 'args': {'ticker': 'TGT'}, 'id': 'call_Wo2SwZE89h1mPyPUOUj1Eu25', 'type': 'tool_call'}
Tool execution is completed. Back to the model!


In [18]:
for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

What is the EBITDA of Target and its last close price?
================================== Ai Message ==================================
Tool Calls:
  EBITDA (call_eKBYzK5uincTMXvCkv6ByZy5)
 Call ID: call_eKBYzK5uincTMXvCkv6ByZy5
  Args:
    ticker: TGT
  last_close_price (call_Wo2SwZE89h1mPyPUOUj1Eu25)
 Call ID: call_Wo2SwZE89h1mPyPUOUj1Eu25
  Args:
    ticker: TGT
================================= Tool Message =================================
Name: EBITDA

8720000000
================================= Tool Message =================================
Name: last_close_price

104.06
================================== Ai Message ==================================

Target's EBITDA is $8.72 billion, and its last close price is $104.06.


In [19]:
result['messages'][-1].content

"Target's EBITDA is $8.72 billion, and its last close price is $104.06."

In [20]:
messages = [HumanMessage(content="Compare total revenue of Amazon and Google.")]

result = agent.graph.invoke({'messages': messages})


Calling Tool : {'name': 'total_revenue', 'args': {'ticker': 'AMZN'}, 'id': 'call_o6x9V4nhCZDkkgObiUq55U4L', 'type': 'tool_call'}
Calling Tool : {'name': 'total_revenue', 'args': {'ticker': 'GOOGL'}, 'id': 'call_FI9IybZkmPq0rQJJibgSSt60', 'type': 'tool_call'}
Tool execution is completed. Back to the model!


In [21]:
result['messages'][-1].content

'The total revenue of Amazon (AMZN) is $637.959 billion, while the total revenue of Google (GOOGL) is $350.018 billion.'